# Langchain + GPT3.5 For Dependency Extraction
First we are going to run this on a small subset of Flink as the whole thing will cost a lot of money.  We will start with a simple reduced structure found in /flink-reduced

### Step 1: Download reqs, load OPENAI_API_KEY from env

In [6]:
# ! pip install openai tiktoken chromadb langchain
from dotenv import load_dotenv, find_dotenv
load_dotenv()

True

### Step 2: Load Documents

In [119]:
import tiktoken
import os 
import re
path = ".\\flink-1.17.1"

def remove_comments(code):
    # Removing /* ... */ comments
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    # Removing // comments
    code = re.sub(r'//.*', '', code)
    return code

def read_file(file_path):
  with open(file_path, "r", encoding="utf-8") as f:
    return f.read()
  
def load_documents():
  documents = []
  encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-instruct")

  for dir_path, _, file_names in os.walk(path):
    for file_name in file_names:
      if file_name.endswith(".java"):
        content = read_file(dir_path + "\\" + file_name)
        content = remove_comments(content)

        while len(encoding.encode(content)) > 3000:
          content = content[:4000]
          
        documents.append({"file_path": dir_path + "\\" + file_name, "source": content})
  return documents


documents = load_documents()
len(documents)

13218

In [80]:
print(documents[0])
print(documents[1])
print(documents[2])

{'file_path': '.\\flink-1.17.1\\flink-annotations\\src\\main\\java\\org\\apache\\flink\\FlinkVersion.java', 'source': '\n\npackage org.apache.flink;\n\nimport org.apache.flink.annotation.Public;\n\nimport java.util.Arrays;\nimport java.util.LinkedHashSet;\nimport java.util.Map;\nimport java.util.Optional;\nimport java.util.Set;\nimport java.util.function.Function;\nimport java.util.stream.Collectors;\nimport java.util.stream.Stream;\n\n\n@Public\npublic enum FlinkVersion {\n\n    \n    \n    \n    v1_3("1.3"),\n    v1_4("1.4"),\n    v1_5("1.5"),\n    v1_6("1.6"),\n    v1_7("1.7"),\n    v1_8("1.8"),\n    v1_9("1.9"),\n    v1_10("1.10"),\n    v1_11("1.11"),\n    v1_12("1.12"),\n    v1_13("1.13"),\n    v1_14("1.14"),\n    v1_15("1.15"),\n    v1_16("1.16"),\n    v1_17("1.17");\n\n    private final String versionStr;\n\n    FlinkVersion(String versionStr) {\n        this.versionStr = versionStr;\n    }\n\n    @Override\n    public String toString() {\n        return versionStr;\n    }\n\n  

### Step 3: Define Templates

In [120]:
from langchain.prompts import PromptTemplate
   
initial_prompt = PromptTemplate(
    template="""
    Instruction: You will only return valid JSON. Given the following code, extract any internal dependencies. 
    Output must be a valid JSON array of strings. For the given code you must 
    determine what external files or packages it depends on, and return them.              
    File Path: {file_path}
    Code: {source} 
    Answer in valid JSON: \n\n###\n\n""",
    input_variables=["file_path", "source"]
)

retry_prompt = PromptTemplate(
    template="""
    The previous response, "{response}" was not valid JSON. Please try again. 
    Instruction: You will only return valid JSON. Given the following code, extract any internal dependencies. 
    Output must be a valid JSON array of strings. For the given code you must 
    determine what external files or packages it depends on, and return them.                    
    File Path: {file_path}
    Code: {source} 
    Answer in valid, unformatted JSON: \n\n###\n\n""",
    input_variables=["response", "file_path", "source"]
)

### Step 4: Define response extraction stages

In [121]:
import json
from langchain.llms import OpenAI

output = []
parse_fails = 0
retry_fails = 0
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct", frequency_penalty=0, presence_penalty=0, top_p=0.9, max_tokens=1000)

def invoke_model(document, prev_response=None):
    file_path = document["file_path"]
    source = document["source"]
    is_retry = prev_response is not None

    if is_retry:
        formatted = retry_prompt.format(file_path=file_path, source=source, response=prev_response)
    else: 
        formatted = initial_prompt.format(file_path=file_path, source=source)

    response = llm.invoke(formatted)
    response = response.replace("\n", "").replace(" ", "")

    success = handle_response(file_path, response, is_retry)
    if success or (not success and is_retry): 
        return
    else:
        invoke_model(document, prev_response=response)

            
def handle_response(file_path, response, is_retry):
    buffered_output = []
    try: 
        parsed = json.loads(response)

        for dependency in parsed:
            if isinstance(dependency, str):
                buffered_output.append((file_path, dependency))
            else:
                handle_parse_fail(response, "Dependency was not a string", is_retry)
                return False

    except json.JSONDecodeError as e:
        handle_parse_fail(response, "Invalid JSON", is_retry)
        return False
    
    finally:
        for item in buffered_output:
            output.append(item)
        return True


def handle_parse_fail(response, msg, is_retry):
    global retry_fails, parse_fails

    if is_retry:
        retry_fails += 1
        msg_type = "Retry"
    else:
        parse_fails += 1
        msg_type = "Parse"
        
    print(f"{msg_type} fail ({msg}): {response}")

### Step 5: Process in batches

In [129]:
output = []
batch_start = 0


In [133]:
import pickle
from IPython.display import display, clear_output

batch_size = 200
batch_end = batch_start + batch_size
batch = documents[batch_start:batch_end]

for i, document in enumerate(batch):
    try:
        invoke_model(document)
    except Exception as e:
        modified_doc = document["source"][:len(document["source"]) // 2]
    clear_output(wait=True)
    display(f"Iteration: {i}")
        
batch_start = batch_end

print(f"Batch start: {batch_end - batch_size}")
print(f"Batch size: {batch_size}")
print(f"Parse fails: {parse_fails}")
print(f"Retry fails: {retry_fails}")
print(f"Output length: {len(output)}")

with open(f"bin/deps_{batch_end - batch_size}_{batch_end}.pkl", "wb") as f:
    pickle.dump(output, f)
    print("Saved output")

print(f"Start for next run is {batch_end}")

'Iteration: 199'

Batch start: 400
Batch size: 200
Parse fails: 59
Retry fails: 0
Output length: 11274
Saved output
Start for next run is 600


In [ ]:
for i, document in enumerate(documents):
    len(encoding.encode(document.source)) 
    while len(encoding.encode(content)) > 3000:
        content = content[:4000]
        
    documents.append({"file_path": dir_path + "\\" + file_name, "source": content})

In [86]:
for x,y in output:
    print(x,y)

.\flink-1.17.1\flink-clients\src\main\java\org\apache\flink\client\cli\CliArgsException.java java.lang.Exception
.\flink-1.17.1\flink-clients\src\main\java\org\apache\flink\client\cli\ClientOptions.java org.apache.flink.annotation.PublicEvolving
.\flink-1.17.1\flink-clients\src\main\java\org\apache\flink\client\cli\ClientOptions.java org.apache.flink.configuration.ConfigOption
.\flink-1.17.1\flink-clients\src\main\java\org\apache\flink\client\cli\ClientOptions.java org.apache.flink.configuration.ConfigOptions
.\flink-1.17.1\flink-clients\src\main\java\org\apache\flink\client\cli\ClientOptions.java org.apache.flink.configuration.DeploymentOptions
.\flink-1.17.1\flink-clients\src\main\java\org\apache\flink\client\cli\ClientOptions.java org.apache.flink.configuration.description.Description
.\flink-1.17.1\flink-clients\src\main\java\org\apache\flink\client\cli\ClientOptions.java org.apache.flink.configuration.description.TextElement
.\flink-1.17.1\flink-clients\src\main\java\org\apache\fl